In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import os

## Get Layout file

In [81]:
Layout = pd.read_excel('K:\CPDiary\Data\BaxtersMVI\TrialLayout.xlsx',index_col='Plot')

In [82]:
NPFiles = []
TDRFiles = []
mydir = 'K:\CPDiary\Data\BaxtersMVI\TestData'
for file in os.listdir(mydir):
    if file.endswith('.NP1'):
        NPFiles.append(os.path.join(mydir, file))
    if file.endswith('.NP2'):
        NPFiles.append(os.path.join(mydir, file))
    if file.endswith('.TDR'):
        TDRFiles.append(os.path.join(mydir, file))

In [83]:
#Set up blank dataframe to take data
ColumnLables = ['Plot','Date','D4','D3','D2','D1']
TabIndex = pd.MultiIndex.from_arrays([[],[]],names=['Plot','Date'])
SoilWaterData = pd.DataFrame(columns=ColumnLables[2:],index = TabIndex)
#Read each file
for ObsTable in NPFiles:
    #Read in file and find which lines have legit data
    HeaderRowsToSkip = 30
    DataTab = pd.read_csv(ObsTable,skiprows = range(HeaderRowsToSkip),header=None,delim_whitespace=True)
    DataTab.dropna(inplace=True)
    DataTab.drop([0,1,3,5],axis=1,inplace=True)
    DataTab.columns = ColumnLables
    DataTab.ix[:,1] = pd.to_datetime(DataTab.ix[:,1],format='%m/%d/%y')
    SoilWaterData = SoilWaterData.append(DataTab)
SoilWaterData.ix[:,'Plot'] = np.int64(SoilWaterData.ix[:,'Plot'] )
SoilWaterData = SoilWaterData.where(SoilWaterData.ix[:,'Plot']<25).dropna()
SoilWaterData.ix[:,'Zone'] = [Layout.ix[int(x),'Zone'] for x in SoilWaterData.Plot]
SoilWaterData.ix[:,'Irrig'] = [Layout.ix[int(x),'Irrig'] for x in SoilWaterData.Plot]
SoilWaterData.ix[:,'Rep'] = [Layout.ix[int(x),'Rep'] for x in SoilWaterData.Plot]
SoilWaterData.set_index(['Plot','Date'],inplace=True,drop=False)

#Add in the TDR Data
SoilWaterData.ix[:,'D0'] = np.nan
for TDRTable in TDRFiles:
    TDRData = pd.read_csv(TDRTable,header=None).ix[:,[2,11,3]]
    TDRData.columns = ['Plot','Date','D0']
    TDRData = TDRData.drop_duplicates(subset=['Plot','Date'])
    TDRData.ix[:,'Date'] = pd.to_datetime(TDRData.ix[:,'Date'], format = '%d%b%Y')
    TDRData.set_index(['Plot','Date'],inplace=True)
    SoilWaterData.update(TDRData)

In [84]:
Means = SoilWaterData.groupby(by=['Date', 'Zone', 'Irrig']).mean()

In [87]:
Means

D1         D2         D3         D4       Plot  \
Date       Zone Irrig                                                          
2016-11-28 High IRR    26.000000  28.650000  28.350000  24.575000  20.500000   
                NP     24.125000  27.275000  33.775000  27.325000  20.500000   
           Low  IRR    25.700000  26.650000  30.175000  30.300000   4.500000   
                NP     24.450000  26.600000  31.775000  31.175000   4.500000   
           Mid  IRR    24.825000  29.075000  29.225000  22.600000  12.500000   
                NP     26.550000  28.100000  30.900000  29.250000  12.500000   
2016-12-05 High IRR    24.850000  27.525000  26.375000  23.500000  20.500000   
                NP     22.950000  25.875000  33.000000  26.300000  20.500000   
           Low  IRR    25.000000  25.225000  28.900000  29.700000   4.500000   
                NP     23.075000  24.250000  30.550000  30.800000   4.500000   
           Mid  IRR    20.410000  27.190000  21.930000  22.860000  12.000000   
                NP     24.975000  26.900000  29.600000  28.525000  12.500000   
2016-12-12 High IRR    33.250000  31.850000  29.575000  24.125000  20.500000   
                NP     32.275000  31.725000  33.750000  26.175000  20.500000   
           Low  IRR    35.025000  32.400000  33.225000  30.050000   4.500000   
                NP     33.025000  32.925000  34.275000  31.350000   4.500000   
           Mid  IRR    35.133333  33.600000  29.266667  24.333333  11.333333   
                NP     34.733333  28.700000  30.233333  26.600000  11.666667   
2016-12-19 High IRR    28.800000  30.400000  29.125000  23.800000  20.500000   
                NP     27.375000  29.950000  33.750000  26.850000  20.500000   
           Low  IRR    29.475000  30.325000  32.275000  29.625000   4.500000   
                NP     27.666667  31.933333  33.066667  31.066667   5.666667   
           Mid  IRR    29.075000  30.950000  29.350000  21.825000  12.500000   
                NP     29.750000  29.500000  31.375000  28.875000  12.500000   
2016-12-29 High IRR    24.600000  27.275000  27.650000  23.025000  20.500000   
                NP     22.225000  26.225000  32.825000  26.275000  20.500000   
           Low  IRR    26.350000  27.450000  30.725000  29.975000   4.500000   
                NP     26.050000  29.875000  33.675000  32.300000   4.500000   
           Mid  IRR    24.225000  28.100000  28.150000  21.225000  12.500000   
                NP     25.050000  27.300000  29.925000  28.825000  12.500000   
2017-01-09 High IRR    18.525000  21.825000  24.175000  22.075000  20.500000   
                NP     16.280000  19.600000  29.840000  24.260000  20.800000   
           Low  IRR    19.925000  20.050000  27.275000  28.875000   4.500000   
                NP     19.000000  20.666667  29.666667  30.200000   5.666667   
           Mid  IRR    18.525000  21.325000  24.950000  19.725000  12.500000   
                NP     19.775000  21.700000  25.825000  26.925000  12.500000   
2017-01-16 High IRR    17.300000  19.400000  22.325000  21.250000  20.500000   
                NP     14.650000  17.950000  28.675000  23.175000  20.500000   
           Low  IRR    17.500000  16.600000  25.100000  28.750000   4.500000   
                NP     16.833333  16.466667  27.766667  30.133333   5.666667   
           Mid  IRR    17.200000  19.100000  22.650000  18.725000  12.500000   
                NP     18.300000  19.650000  23.725000  26.125000  12.500000   
2017-01-23 High IRR    16.575000  17.850000  20.450000  20.600000  20.500000   
                NP     14.950000  15.750000  26.625000  21.950000  20.500000   
           Low  IRR    16.550000  14.800000  21.900000  27.925000   4.500000   
                NP     16.775000  13.325000  22.925000  29.550000   4.500000   
           Mid  IRR    16.675000  17.550000  20.800000  17.600000  12.500000   
                NP     17.400000  18.075000  21.050000  24.575000  12.500000   
2017-01-30 High IRR    15.7500